# FrozenLake之策略循环(Policy Iteration)

In [1]:
# 载入相关套件
import numpy as np
import gym

In [2]:
env = gym.make('FrozenLake-v1')
env.reset()

0

In [6]:
nS = env.observation_space.n
nA = env.action_space.n

In [7]:
env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

## 策略评估函数

In [9]:
# 策略评估函数
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    # 状态值函数初始化
    V = np.zeros(nS)
    V1 = np.copy(V)
    while True:
        delta = 0
        # 更新每个状态值的函数
        for s in range(nS):
            v = 0
            # 计算每个行动后的状态值函数
            for a, action_prob in enumerate(policy[s]):
                # 取得所有可能的下一状态值
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 状态值函数公式，依照所有可能的下一状态值函数加总 
                    v += action_prob * prob * (reward + 
                                   discount_factor * V[next_state])
            # 比较更新前后的差值，取最大值
            delta = max(delta, np.abs(v - V[s]))
            V1[s] = v
        V = np.copy(V1)
        # 若最大差值 < 门槛值，则停止评估
        if delta < theta:
            break
    return np.array(V)

In [11]:
# 随机策略，机率均等
random_policy = np.ones([env.observation_space.n, env.action_space.n]) / env.action_space.n
# 评估
v = policy_eval(random_policy, env)
print("状态值函数:")
print(v.reshape((int(nS ** 0.5), int(nS ** 0.5))))

状态值函数:
[[0.01390102 0.01160346 0.02092871 0.01045776]
 [0.0162259  0.         0.04074201 0.        ]
 [0.03479274 0.08816183 0.14204605 0.        ]
 [0.         0.1758143  0.43928541 0.        ]]


## 策略改善函数

In [15]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    # 计算行动值函数
    def one_step_lookahead(state, V):
        A = np.zeros(nA)
        for a in range(nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # 一开始采随机策略，往上/下/左/右走的机率(π)均等
    policy = np.ones([nS, nA]) / nA
    
    while True:
        # 策略评估
        V = policy_eval_fn(policy, env, discount_factor)
        
        # 若要改变策略，会设定 policy_stable = False
        policy_stable = True
        
        for s in range(nS):
            # 依 P 选择最佳行动
            chosen_a = np.argmax(policy[s])
            
            # 计算下一步的行动值函数
            action_values = one_step_lookahead(s, V)
            # 选择最佳行动
            best_a = np.argmax(action_values)
            
            # 贪婪策略：若有新的最佳行动，修改行动策略
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(nA)[best_a]
        
        # 如果已无较佳行动策略，则回传策略及状态值函数
        if policy_stable:
            return policy, V

In [16]:
# 执行策略循环
policy, v = policy_improvement(env)

In [19]:
# 显示结果
print("策略机率分配:")
print(policy)
print("")

print("4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), (int(nS ** 0.5), int(nS ** 0.5))))
print("")

print("4x4 状态值函数:")
print(v.reshape((int(nS ** 0.5), int(nS ** 0.5))))

策略机率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 3]
 [0 0 0 0]
 [3 1 0 0]
 [0 2 1 0]]

4x4 状态值函数:
[[0.8232943  0.82321549 0.82315953 0.8231305 ]
 [0.82331142 0.         0.52924366 0.        ]
 [0.8233444  0.82339085 0.76458368 0.        ]
 [0.         0.88225455 0.94112542 0.        ]]
